In [1]:
import pandas as pd

In [2]:
#units = "F"
#up_thresh = 86 #F #30 #C
#dn_thresh = 50 #F #10 #C

units = "C"
up_thresh = 30 #C
dn_thresh = 10 #C

field_data = pd.read_csv( "../data/Masternote Washburn.csv")
sow_date = pd.to_datetime(field_data['Date Plot Planted [MM/DD/YY]'].unique()[0])
met_data = pd.read_csv("../data/2020_Bradford_MOH1_weather.csv")

In [3]:
#calculate GDD Based on https://ndawn.ndsu.nodak.edu/help-corn-growing-degree-days.html

#convert to F
#(Celsius * 1.8) + 32
temp_data = met_data[["Date_key","Temperature [C]"]].dropna().reset_index(drop=True).copy()
if units == "F":
    temp_data["Temperature [F]"] = ((temp_data["Temperature [C]"]*1.8) + 32)
    temp_data.drop(columns="Temperature [C]", inplace=True)
    
#get max and min temps
temp_data["Date_time"] = pd.to_datetime(temp_data["Date_key"])
temp_data["Date"] = pd.to_datetime(temp_data["Date_time"].dt.date)
gdd = temp_data.pivot_table(index=["Date"], values = "Temperature ["+units+"]", aggfunc=['min',"max"])
gdd.columns = [x[0]+" "+x[1] for x in gdd.columns]

#if min and/or max < dn_thresh set to dn_thresh
gdd.loc[gdd["min Temperature ["+units+"]"] < dn_thresh, "min Temperature ["+units+"]"] = dn_thresh
gdd.loc[gdd["max Temperature ["+units+"]"] < dn_thresh, "max Temperature ["+units+"]"] = dn_thresh

#if min and/or max > up_thresh set to up_thresh
gdd.loc[gdd["min Temperature ["+units+"]"] > up_thresh, "min Temperature ["+units+"]"] = up_thresh
gdd.loc[gdd["max Temperature ["+units+"]"] > up_thresh, "max Temperature ["+units+"]"] = up_thresh

#calculate mean temp and subtract dn_thresh from it
gdd["mean temperature ["+units+"]"] = (gdd["max Temperature ["+units+"]"] + gdd["min Temperature ["+units+"]"])/2
gdd["GDD ["+units+"]"] = gdd["mean temperature ["+units+"]"] - dn_thresh

#calculate cummulative GDD
gdd = gdd.sort_index() # make sure in correct order
gdd["DAP"] = 1 #add DAP - days after planting
gdd["DAP"] = gdd["DAP"].cumsum()
gdd["Cum_GDD ["+units+"]"] = gdd["GDD ["+units+"]"].cumsum()

#save to file
gdd.to_csv("../data/GDDs_DAPs.csv")
gdd

,min Temperature [C],max Temperature [C],mean temperature [C],GDD [C],DAP,Cum_GDD [C]
Date,,,,,,
2020-06-02,18.28,30.00,24.140,14.140,1,14.140
2020-06-03,18.89,30.00,24.445,14.445,2,28.585
2020-06-04,17.89,28.56,23.225,13.225,3,41.810
2020-06-05,19.39,30.00,24.695,14.695,4,56.505
2020-06-06,19.61,30.00,24.805,14.805,5,71.310
...,...,...,...,...,...,...
2020-10-18,10.00,18.28,14.140,4.140,139,1730.830
2020-10-19,10.00,10.00,10.000,0.000,140,1730.830
2020-10-20,10.00,11.00,10.500,0.500,141,1731.330


In [4]:
#calculate GDD to silk/pollen
flowering = field_data[['Pedigree', 'Plot', 'Pollen Date [MM/DD/YY]', 'Silk Date [MM/DD/YY]']].copy()
flowering["Pollen Date"] = pd.to_datetime(flowering["Pollen Date [MM/DD/YY]"])
flowering['Silk Date'] = pd.to_datetime(flowering['Silk Date [MM/DD/YY]'])
flowering.drop(columns=['Pollen Date [MM/DD/YY]', 'Silk Date [MM/DD/YY]'], inplace=True)
#remove NaN dates
flowering = flowering.dropna()
print(len(flowering))
#add in DAP and GDD
flowering = flowering.merge(gdd[["DAP", "Cum_GDD ["+units+"]"]], left_on="Pollen Date", right_index=True, how="left")
flowering = flowering.rename(columns={"DAP":"Pollen_DAP", "Cum_GDD ["+units+"]":"Pollen_Cum_GDD ["+units+"]"})
flowering = flowering.merge(gdd[["DAP", "Cum_GDD ["+units+"]"]], left_on="Silk Date", right_index=True, how="left")
flowering = flowering.rename(columns={"DAP":"Silk_DAP", "Cum_GDD ["+units+"]":"Silk_Cum_GDD ["+units+"]"})
print(len(flowering))

flowering.to_csv("../data/flowering_time_GDD_DAP.csv")
print(flowering[['Pollen_DAP',"Pollen_Cum_GDD ["+units+"]", 'Silk_DAP', "Silk_Cum_GDD ["+units+"]"]].min())
print(flowering[['Pollen_DAP',"Pollen_Cum_GDD ["+units+"]", 'Silk_DAP', "Silk_Cum_GDD ["+units+"]"]].max())
flowering

1531
1531
Pollen_DAP             46.000
Pollen_Cum_GDD [C]    665.745
Silk_DAP               47.000
Silk_Cum_GDD [C]      682.495
dtype: float64
Pollen_DAP              69.000
Pollen_Cum_GDD [C]     993.125
Silk_DAP                79.000
Silk_Cum_GDD [C]      1131.240
dtype: float64


,Pedigree,Plot,Pollen Date,Silk Date,Pollen_DAP,Pollen_Cum_GDD [C],Silk_DAP,Silk_Cum_GDD [C]
0,W10004_0121 X PHZ51,1,2020-07-28,2020-07-31,57,839.155,60,880.930
1,W10004_0220 X PHZ51,2,2020-07-29,2020-07-29,58,854.905,58,854.905
2,W10004_0619 X PHZ51,3,2020-07-28,2020-07-27,57,839.155,56,824.490
3,W10004_0072 X PHZ51,4,2020-07-27,2020-07-27,56,824.490,56,824.490
4,W10004_1032 X PHZ51,5,2020-07-25,2020-07-25,54,792.185,54,792.185
...,...,...,...,...,...,...,...,...
1589,PHB47 X PHK76,1590,2020-07-24,2020-07-25,53,775.770,54,792.185
1590,TX714 X PHZ51,1591,2020-07-26,2020-07-27,55,808.740,56,824.490
1591,PHB47 X PHZ51,1592,2020-07-25,2020-07-26,54,792.185,55,808.740
1592,LH195 X PHN82,1593,2020-07-28,2020-07-31,57,839.155,60,880.930


In [5]:
mean_DAP = flowering[["Pollen_DAP", "Silk_DAP"]].values.flatten().mean()
std_DAP = flowering[["Pollen_DAP", "Silk_DAP"]].values.flatten().std()
print("Mean DAP: ", mean_DAP)
print("Flowering DAP, Mean +/- Std: ", mean_DAP-std_DAP,",", mean_DAP+std_DAP)

mean_GDD = flowering[["Pollen_Cum_GDD ["+units+"]", "Silk_Cum_GDD ["+units+"]"]].values.flatten().mean()
std_GDD = flowering[["Pollen_Cum_GDD ["+units+"]", "Silk_Cum_GDD ["+units+"]"]].values.flatten().std()
print("Mean Cum_GDD ["+units+"]: ", mean_GDD)
print("Flowering Cum_GDD ["+units+"], Mean +/- Std: ", mean_GDD-std_GDD,",", mean_GDD+std_GDD)

Mean DAP:  54.2073807968648
Flowering DAP, Mean +/- Std:  51.19470817271569 , 57.2200534210139
Mean Cum_GDD [C]:  794.187393860222
Flowering Cum_GDD [C], Mean +/- Std:  750.1523934725153 , 838.2223942479286
